# Giesekus Creep: Strain Response Under Constant Stress

## Learning Objectives

1. Fit the Giesekus model to real mucus creep compliance data
2. Understand creep behavior in viscoelastic fluids
3. Extract retardation spectrum from creep response
4. Compare Giesekus creep to Maxwell (single exponential)
5. Use ODE-based simulation via diffrax

## Prerequisites

- Basic RheoJAX usage (basic/01-maxwell-fitting.ipynb)
- Bayesian inference fundamentals (bayesian/01-bayesian-basics.ipynb)

## Runtime

- Fast demo (NUM_CHAINS=1, NUM_SAMPLES=500): ~3-5 minutes
- Full run (NUM_CHAINS=4, NUM_SAMPLES=2000): ~15-20 minutes

## 1. Setup

In [ ]:
# Google Colab setup
import sys

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax
    import os
    os.environ["JAX_ENABLE_X64"] = "true"
    print("RheoJAX installed successfully.")

In [ ]:
# Imports
%matplotlib inline
import json
import os
import time
import warnings

import arviz as az
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

from rheojax.core.jax_config import safe_import_jax, verify_float64
from rheojax.models.giesekus import GiesekusSingleMode

jax, jnp = safe_import_jax()
verify_float64()

warnings.filterwarnings("ignore", category=FutureWarning)
print(f"JAX version: {jax.__version__}")
print(f"Devices: {jax.devices()}")

## 2. Theory: Creep in Viscoelastic Fluids

### The Experiment

In a creep test, we apply a constant stress σ₀ at t = 0 and measure the strain γ(t) over time.

### Creep Compliance

The creep compliance is defined as:
$$
J(t) = \frac{\gamma(t)}{\sigma_0}
$$

For a Maxwell fluid (linear viscoelastic):
$$
J(t) = \frac{1}{G} + \frac{t}{\eta} = \frac{1}{G}\left(1 + \frac{t}{\lambda}\right)
$$

where G = η/λ is the elastic modulus.

### Giesekus Creep

For the Giesekus model, the creep response is **nonlinear** due to the τ·τ term. At low stresses (σ₀ << G), the response approaches Maxwell. At high stresses, the nonlinearity accelerates creep.

The strain response is computed by solving the tensorial ODE:
$$
\boldsymbol{\tau} + \lambda \stackrel{\nabla}{\boldsymbol{\tau}} + \frac{\alpha \lambda}{\eta_p} \boldsymbol{\tau} \cdot \boldsymbol{\tau} = 2\eta_p \mathbf{D}
$$

subject to constant shear stress τ_xy = σ₀.

### Key Features

| Time Scale | Behavior |
|------------|----------|
| t << λ | Elastic response: γ ≈ σ₀/G |
| t >> λ | Viscous flow: γ ≈ σ₀·t/η |
| t ~ λ | Transition: viscoelastic creep |

## 3. Load Data

We use mucus creep compliance data. Mucus is a biopolymer gel that exhibits complex viscoelastic behavior.

In [ ]:
# Load mucus creep data
data_path = os.path.join("..", "data", "creep", "biological", "creep_mucus_data.csv")

# Tab-separated with header
raw = np.loadtxt(data_path, delimiter="\t", skiprows=1)
time_data = raw[:, 0]  # Time (s)
J_data = raw[:, 1]  # Creep compliance (1/Pa)

print(f"Loaded {len(time_data)} data points")
print(f"Time range: [{time_data.min():.2f}, {time_data.max():.1f}] s")
print(f"J range: [{J_data.min():.2e}, {J_data.max():.2e}] 1/Pa")

In [ ]:
# Plot raw data
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Linear scale
ax1.plot(time_data, J_data, "ko", markersize=5)
ax1.set_xlabel("Time [s]")
ax1.set_ylabel("Creep compliance J(t) [1/Pa]")
ax1.set_title("Mucus Creep (Linear Scale)")
ax1.grid(True, alpha=0.3)

# Log-log scale
ax2.loglog(time_data, J_data, "ko", markersize=5)
ax2.set_xlabel("Time [s]")
ax2.set_ylabel("Creep compliance J(t) [1/Pa]")
ax2.set_title("Mucus Creep (Log-Log Scale)")
ax2.grid(True, alpha=0.3, which="both")

plt.tight_layout()
display(fig)
plt.close(fig)

## 4. NLSQ Fitting

We fit the Giesekus model to the creep compliance data.

In [ ]:
# Create and fit Giesekus model
model = GiesekusSingleMode()

# For creep, we need to specify the applied stress
# Estimate from compliance: J = γ/σ, so we need σ
# Assume σ = 1 Pa for compliance normalization
sigma_applied = 1.0  # Pa

t0 = time.time()
model.fit(time_data, J_data, test_mode="creep", sigma_applied=sigma_applied, method='scipy')
t_nlsq = time.time() - t0

print(f"NLSQ fit time: {t_nlsq:.2f} s")
print(f"\nFitted parameters:")
param_names = ["eta_p", "lambda_1", "alpha", "eta_s"]
for name in param_names:
    val = model.parameters.get_value(name)
    print(f"  {name:10s} = {val:.4g}")

# Derived quantities
eta_p = model.parameters.get_value("eta_p")
eta_s = model.parameters.get_value("eta_s")
lambda_1 = model.parameters.get_value("lambda_1")
alpha = model.parameters.get_value("alpha")

eta_0 = eta_p + eta_s
G_0 = eta_p / lambda_1

print(f"\nDerived quantities:")
print(f"  η₀ = η_p + η_s = {eta_0:.2f} Pa·s")
print(f"  G₀ = η_p/λ = {G_0:.2f} Pa")
print(f"  J₀ = 1/G₀ = {1/G_0:.4e} 1/Pa")

In [ ]:
# Plot fit with data
time_fine = np.logspace(
    np.log10(time_data.min()),
    np.log10(time_data.max()),
    200,
)
J_pred = model.predict(time_fine, test_mode="creep", sigma_applied=sigma_applied)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Linear scale
ax1.plot(time_data, J_data, "ko", markersize=5, label="Data")
ax1.plot(time_fine, J_pred, "-", lw=2, color="C0", label="Giesekus fit")
ax1.set_xlabel("Time [s]")
ax1.set_ylabel("Creep compliance J(t) [1/Pa]")
ax1.set_title("Creep Fit (Linear Scale)")
ax1.legend()
ax1.grid(True, alpha=0.3)

# Log-log scale
ax2.loglog(time_data, J_data, "ko", markersize=5, label="Data")
ax2.loglog(time_fine, J_pred, "-", lw=2, color="C0", label="Giesekus fit")
ax2.set_xlabel("Time [s]")
ax2.set_ylabel("Creep compliance J(t) [1/Pa]")
ax2.set_title("Creep Fit (Log-Log Scale)")
ax2.legend()
ax2.grid(True, alpha=0.3, which="both")

plt.tight_layout()
display(fig)
plt.close(fig)

## 5. Bayesian Inference

In [ ]:
# Bayesian inference with NLSQ warm-start
initial_values = {
    name: model.parameters.get_value(name)
    for name in param_names
}
print("Warm-start values:")
for k, v in initial_values.items():
    print(f"  {k}: {v:.4g}")

# Fast demo config
NUM_WARMUP = 200
NUM_SAMPLES = 500
NUM_CHAINS = 1
# NUM_WARMUP = 1000; NUM_SAMPLES = 2000; NUM_CHAINS = 4  # production

t0 = time.time()
result = model.fit_bayesian(
    time_data,
    J_data,
    test_mode="creep",
    sigma_applied=sigma_applied,
    num_warmup=NUM_WARMUP,
    num_samples=NUM_SAMPLES,
    num_chains=NUM_CHAINS,
    initial_values=initial_values,
    seed=42,
)
t_bayes = time.time() - t0
print(f"\nBayesian inference time: {t_bayes:.1f} s")

In [ ]:
# Convergence diagnostics
diag = result.diagnostics

print("Convergence Diagnostics")
print("=" * 55)
print(f"{'Parameter':>12s}  {'R-hat':>8s}  {'ESS':>8s}  {'Status':>8s}")
print("-" * 55)

for p in param_names:
    r_hat = diag.get("r_hat", {}).get(p, float("nan"))
    ess = diag.get("ess", {}).get(p, float("nan"))
    status = "PASS" if (r_hat < 1.05 and ess > 100) else "CHECK"
    print(f"{p:>12s}  {r_hat:8.4f}  {ess:8.0f}  {status:>8s}")

n_div = diag.get("divergences", diag.get("num_divergences", 0))
print(f"\nDivergences: {n_div}")

In [ ]:
# ArviZ trace plots
idata = result.to_inference_data()

axes = az.plot_trace(idata, var_names=param_names, figsize=(12, 8))
fig = axes.ravel()[0].figure
fig.suptitle("Trace Plots", fontsize=14, y=1.02)
plt.tight_layout()
display(fig)
plt.close(fig)

In [ ]:
# Forest plot
axes = az.plot_forest(
    idata,
    var_names=param_names,
    combined=True,
    hdi_prob=0.95,
    figsize=(10, 5),
)
fig = axes.ravel()[0].figure
plt.tight_layout()
display(fig)
plt.close(fig)

## 6. Posterior Predictive Check

In [ ]:
# Posterior predictive with 95% credible intervals
posterior = result.posterior_samples
n_draws = min(100, len(posterior["alpha"]))  # Fewer draws for ODE-based prediction

time_pred = np.logspace(
    np.log10(time_data.min()),
    np.log10(time_data.max()),
    50,  # Fewer points for ODE
)

pred_samples = []
for i in range(n_draws):
    # Set parameters from posterior
    for name in param_names:
        model.parameters.set_value(name, float(posterior[name][i]))
    
    # Predict using the correct API
    pred_i = model.predict(time_pred, test_mode="creep", sigma_applied=sigma_applied)
    pred_samples.append(np.array(pred_i))

pred_samples = np.array(pred_samples)
pred_median = np.median(pred_samples, axis=0)
pred_lo = np.percentile(pred_samples, 2.5, axis=0)
pred_hi = np.percentile(pred_samples, 97.5, axis=0)

fig, ax = plt.subplots(figsize=(10, 6))
ax.fill_between(time_pred, pred_lo, pred_hi, alpha=0.3, color="C0", label="95% CI")
ax.loglog(time_pred, pred_median, "-", lw=2, color="C0", label="Posterior median")
ax.loglog(time_data, J_data, "ko", markersize=5, label="Data")
ax.set_xlabel("Time [s]")
ax.set_ylabel("Creep compliance J(t) [1/Pa]")
ax.set_title("Posterior Predictive Check")
ax.legend()
ax.grid(True, alpha=0.3, which="both")
plt.tight_layout()
display(fig)
plt.close(fig)

## 7. Parameter Summary

In [ ]:
# NLSQ vs Bayesian comparison
print("Parameter Comparison: NLSQ vs Bayesian")
print("=" * 70)
print(f"{'Param':>12s}  {'NLSQ':>12s}  {'Bayes median':>14s}  {'95% CI':>26s}")
print("-" * 70)

# Reset NLSQ values first
for name, val in initial_values.items():
    model.parameters.set_value(name, val)

for name in param_names:
    nlsq_val = initial_values[name]
    bayes_samples = posterior[name]
    median = float(np.median(bayes_samples))
    lo = float(np.percentile(bayes_samples, 2.5))
    hi = float(np.percentile(bayes_samples, 97.5))
    print(f"{name:>12s}  {nlsq_val:12.4g}  {median:14.4g}  [{lo:.4g}, {hi:.4g}]")

## 8. Save Results

In [ ]:
# Save results
output_dir = os.path.join("..", "outputs", "giesekus", "creep")
os.makedirs(output_dir, exist_ok=True)

# Save NLSQ point estimates
nlsq_params = initial_values.copy()
with open(os.path.join(output_dir, "nlsq_params_creep.json"), "w") as f:
    json.dump(nlsq_params, f, indent=2)

# Save posterior samples
posterior_dict = {k: np.array(v).tolist() for k, v in posterior.items()}
with open(os.path.join(output_dir, "posterior_creep.json"), "w") as f:
    json.dump(posterior_dict, f)

print(f"Results saved to {output_dir}/")

## Key Takeaways

1. **Creep compliance** J(t) = γ(t)/σ₀ characterizes the strain response under constant stress.

2. **Short-time behavior** (t << λ) is elastic: γ ≈ σ₀/G (instantaneous elastic strain).

3. **Long-time behavior** (t >> λ) is viscous: γ ≈ σ₀·t/η (steady flow).

4. **Giesekus vs Maxwell**: The quadratic term introduces stress-dependent nonlinearity. At high stresses, Giesekus shows faster creep than Maxwell.

5. **Biological materials** like mucus show complex viscoelastic behavior with multiple relaxation times.

### Experimental Notes

- Creep tests are sensitive to **material aging** and **thixotropy**
- Low stress creep probes **linear regime** (Giesekus ≈ Maxwell)
- High stress creep probes **nonlinear regime** (α effect visible)

### Next Steps

- **NB 06**: Stress relaxation
- **NB 07**: LAOS